<a href="https://colab.research.google.com/github/mohmaed7777/Novozymes-Enzyme-Stability-Prediction/blob/main/NovozymeAmino_Acid_Properties.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!mkdir ~/.kaggle

In [4]:
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c novozymes-enzyme-stability-prediction


 71% 5.00M/7.06M [00:00<00:00, 50.4MB/s]
100% 7.06M/7.06M [00:00<00:00, 66.2MB/s]


In [7]:
! unzip novozymes-enzyme-stability-prediction.zip

Archive:  novozymes-enzyme-stability-prediction.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               
  inflating: train_updates_20220929.csv  
  inflating: wildtype_structure_prediction_af2.pdb  


In [8]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from itertools import product 
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn.ensemble import HistGradientBoostingRegressor



In [9]:
Aminoacids = ['A', 'R', 'N','D', 'C','E','Q','G','H','I',
              'L','K','M','F','P','S','T','W','Y','V']

Blocks =[]

maxSize = 4

for k in range(1, maxSize):
  Blocks.append([''.join(i) for i in product(Aminoacids, repeat= k)])

In [10]:
#data = pd.read_csv('', index_col='seq_id')
#dataMod = pd.read_csv('', index_col='seq_id')

In [11]:
data = pd.read_csv('/content/train.csv',index_col="seq_id")
dataMod = pd.read_csv('/content/train_updates_20220929.csv',index_col="seq_id")

#from https://www.kaggle.com/competitions/novozymes-enzyme-stability-prediction/discussion/356251 Petr1zi0's answer

all_features_nan = dataMod.isnull().all("columns")
data = data.drop(index=dataMod[all_features_nan].index)

swap_ph_tm_indices = dataMod[~all_features_nan].index
data.loc[swap_ph_tm_indices, ["pH", "tm"]] = dataMod.loc[swap_ph_tm_indices, ["pH", "tm"]]

data['seqsize'] = [len(val) for val in data['protein_sequence']]

data = data.fillna(data['pH'].max())

In [12]:
def SplitString(String,ChunkSize):
    '''
    Split a string ChunkSize fragments using a sliding windiow

    Parameters
    ----------
    String : string
        String to be splitted.
    ChunkSize : int
        Size of the fragment taken from the string .

    Returns
    -------
    Splitted : list
        Fragments of the string.

    '''
    try:
        localString=str(String.seq)
    except AttributeError:
        localString=str(String)
      
    if ChunkSize==1:
        Splitted=[val for val in localString]
    
    else:
        nCharacters=len(String)
        Splitted=[localString[k:k+ChunkSize] for k in range(nCharacters-ChunkSize)]
        
    return Splitted

def CountUniqueElements(UniqueElements,String,Processed=False):
    '''
    Calculates the frequency of the unique elements in a splited or 
    processed string. Returns a list with the frequency of the 
    unique elements. 
    
    Parameters
    ----------
    UniqueElements : array,list
        Elements to be analized.
    String : strting
        Sequence data.
    Processed : bool, optional
        Controls if the sring is already splitted or not. The default is False.
    Returns
    -------
    localCounter : array
        Normalized frequency of each unique fragment.
    '''
    
    nUnique = len(UniqueElements)
    localCounter = [0 for k in range(nUnique)]
    
    if Processed:
        ProcessedString = String
    else:
        ProcessedString = SplitString(String,len(UniqueElements[0]))
        
    nSeq = len(ProcessedString)
    UniqueDictionary = {}
    
    for k,val in enumerate(UniqueElements):
        UniqueDictionary[val] = k
    
    for val in ProcessedString:
        
        if val in UniqueElements:
            
            localPosition=UniqueDictionary[val]
            localCounter[localPosition]=localCounter[localPosition]+1
            
    localCounter=[val/nSeq for val in localCounter]
    
    return localCounter

In [13]:
dta0 = np.array([CountUniqueElements(Aminoacids, val) for  val in data['protein_sequence']])
props = pd.read_csv('/content/aaindex1.csv')
feats = np.array(props[Aminoacids])
finalfeats = dta0.dot(feats.T)
seqFeatures = pd.DataFrame(finalfeats, columns = props['description'])
seqFeatures['seq_id'] = data.index
seqFeatures = seqFeatures.set_index('seq_id')

data = pd.concat([data, seqFeatures], axis=1)
data = data.fillna(0)

In [14]:
data.head()

,protein_sequence,pH,data_source,tm,seqsize,"alpha-CH chemical shifts (Andersen et al., 1992)","Hydrophobicity index (Argos et al., 1982)","Signal sequence helical potential (Argos et al., 1982)","Membrane-buried preference parameters (Argos et al., 1982)","Conformational parameter of inner helix (Beghin-Dirkx, 1975)",...,"Weighted domination number using the atomic number (Karkbara-Knisley, 2016)",Average weighted eccentricity based on the the atomic number,Weighted radius based on the atomic number (minimum eccentricity),Weighted diameter based on the atomic number (maximum eccentricity),Total weighted atomic number of the graph (obtained by summing all the atomic,Average weighted atomic number or degree based on atomic number in the graph,Weighted maximum eigenvalue based on the atomic numbers,Weighted minimum eigenvalue based on the atomic numbers,Weighted average eigenvalue based on the atomic numbers,Weighted second smallest eigenvalue of the weighted Laplacian matrix
seq_id,,,,,,,,,,,,,,,,,,,,,
0,AAAAKAAALALLGEAPEVVDIWLPAGWRQPFRVFRLERKGDGVLVG...,7.0,doi.org/10.1038/s41592-020-0801-4,75.7,341,4.282581,0.913812,1.046598,1.138915,0.655308,...,9.976540,15.112293,10.727273,18.120235,28.115232,5.824657,21.543613,-0.270540,8.960390,3.132199
1,AAADGEPLHNEEERAGAGQVGRSLPQESEEQRTGSRPRRRRDLGSR...,7.0,doi.org/10.1038/s41592-020-0801-4,50.5,286,4.320175,0.755979,0.808322,0.888776,0.632028,...,11.034965,17.629909,12.356643,21.367133,31.941024,5.984444,23.136941,-0.295948,9.431892,2.851206
2,AAAFSTPRATSYRILSSAGSGSTRADAPQVRRLHTTRDLLAKDYYA...,7.0,doi.org/10.1038/s41592-020-0801-4,40.5,497,4.341429,0.755614,0.800825,0.880684,0.561288,...,9.820926,15.752694,11.150905,19.323944,29.042887,5.866702,21.542694,-0.390036,8.984998,2.984416
3,AAASGLRTAIPAQPLRHLLQPAPRPCLRPFGLLSVRAGSARRSGLL...,7.0,doi.org/10.1038/s41592-020-0801-4,47.2,265,4.361283,0.867698,0.963019,1.003962,0.592415,...,10.547170,16.299219,11.777358,19.720755,30.804381,6.070683,23.355830,-0.434381,9.536706,3.428283
4,AAATKSGPRRQSQGASVRTFTPFYFLVEPVDTLSVRGSSVILNCSA...,7.0,doi.org/10.1038/s41592-020-0801-4,49.5,1451,4.356871,0.872198,0.933880,0.981757,0.554045,...,10.302550,15.926099,11.444521,19.470710,30.018830,6.072841,23.190112,-0.514836,9.570651,4.052998


In [15]:
Xtrain, Xtest, _, _ = train_test_split(data.index, np.arange(len(data.index)), test_size=0.2, random_state=42)

cols = list(seqFeatures) + ['pH', 'seqsize']

# **Kfold CV:=**

In [16]:
kf = KFold(n_splits=20)
perf = []
Models = []
Scalers = []

for train_index, test_index in kf.split(Xtrain):
  localXtrain, localXtest = Xtrain[train_index], Xtrain[test_index]

  scaler = MinMaxScaler()
  scaler.fit(np.array(data[cols].loc[localXtrain]))
  Scalers.append(scaler)

  trainData = scaler.transform(np.array(data[cols].loc[localXtrain]))
  testData = scaler.transform(np.array(data[cols].loc[localXtest]))

  reg = HistGradientBoostingRegressor(max_iter=2700, learning_rate=0.01, random_state=2458)
  reg.fit(trainData, data['tm'].loc[localXtrain])
  Models.append(reg)

  trainPreds = reg.predict(testData)
  